In [134]:
import requests as rq
from bs4 import BeautifulSoup as bs
import csv
import json

In [135]:
linkedin_url = "https://www.linkedin.com/jobs/search"

In [136]:
query_string = {
    "keywords": "Desenvolvedor Junior", #Title
    "location": "Brazil", # Country 
    "trk": "public_jobs_jobs-search-bar_search-submit" #Linkedin Tracking
}

In [137]:
response = rq.get(linkedin_url, params=query_string) # Accessing the page and verifying the status
response.status_code

200

In [138]:
content = bs(response.content, 'html.parser') # Using BeautifulSoup to convert the HTML page

In [139]:
vacancy_list = content.select(".jobs-search__results-list li") # Getting the list of vacancies in the page
vacancy_list

[<li>
 <div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card" data-column="1" data-entity-urn="urn:li:jobPosting:3748107348" data-row="1" data-search-id="ccSjHFVsYxQrjeTwBv87lg==" data-tracking-id="ctZKJcnR+TqifHUcTwcyPw==">
 <a class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://br.linkedin.com/jobs/view/desenvolvedor-a-jr-remoto-at-plugaa%C3%AA-3748107348?refId=ccSjHFVsYxQrjeTwBv87lg%3D%3D&amp;trackingId=ctZKJcnR%2BTqifHUcTwcyPw%3D%3D&amp;position=1&amp;pageNum=0&amp;trk=public_jobs_jserp-result_search-card">
 <span class="sr-only">
               
         
         Desenvolvedor(a) Jr | Remoto
       
       
           </span>
 </a>
 <div class="search-entity-media">
 <img alt="" class="artdeco-entity-image artdeco-entity-

In [140]:
vacancies_list = []
for vacancy in vacancy_list:
    title = vacancy.select(".base-search-card__title")[0].text.strip() # Title
    # obs.: we could use find() instead of select() in some cases.
    
    company = vacancy.select(".base-search-card__subtitle")[0].text.strip() # Subtitle/Company
    
    location = vacancy.select(".job-search-card__location")[0].text.strip() # Location
    
    url = vacancy.select(".base-card__full-link")[0].get('href') # URL
    
    date = vacancy.select("time")[0].get("datetime") # Date
    
    # Putting the informations in a dict, so we can add this dict on the Vacancies List
    info = {
        "Title": title,
        "Company": company,
        "Location": location,
        "URL": url,
        "Date": date
    }
    vacancies_list.append(info)

In [141]:
vacancies_list_json = json.dumps(vacancies_list) # Putting the list in a text format

In [142]:
with open("vacancies.json", "w") as json_archive:
    json_archive.write(vacancies_list_json) # This way we write the list on a .json archive

In [143]:
# Putting the list in a .csv file
with open('vacancies.csv', mode='w') as csv_file:
    header = ['Title', 'Company', 'Location', 'URL', 'Date']
    generator_csv = csv.DictWriter(csv_file, fieldnames=header)
    generator_csv.writeheader()
    generator_csv.writerows(vacancies_list)